In [6]:
!pip install tokenizers transformers torchmetrics


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import numpy as np
import sklearn
import torch
import os
import pandas as pd
import tqdm

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchmetrics

In [8]:
device = 'cpu'

In [9]:
if not os.path.exists('IMDB-Dataset.csv'):
  !wget -O IMDB-Dataset.csv -q "https://www.dropbox.com/scl/fi/0c7zc2adk1mgwgut5w80w/IMDB-Dataset.csv?rlkey=1drfg4zw36mhu32ndy2ihnygw&dl=1"

In [10]:
df = pd.read_csv('IMDB-Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
text = list(df['review'].str.replace('<br />',''))
labels = np.array(df['sentiment'].map({'negative':0,'positive':1}))

In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Example of how to tokenize text:


In [13]:
seq = text[0][:10]
seq

'One of the'

In [14]:
token_ids = tokenizer(seq)['input_ids']
token_ids

[101, 1448, 1104, 1103, 102]

In [15]:
tokenizer.decode(token_ids+[0,0,0])

'[CLS] One of the [SEP] [PAD] [PAD] [PAD]'